# 決定木アルゴリズム作成

In [50]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pydotplus
from sklearn import datasets
from sklearn import tree
from sklearn.model_selection import train_test_split

## ノードクラス(決定木の基となるクラス)の作成

In [37]:
class demoNode(object):
    def __init__(self, max_depth):
        self.left  = None
        self.right = None
        self.max_depth = max_depth
        self.depth = None
        
    def split_node(self, depth):
        self.depth = depth
        print ("Recursion depth: " + str(self.depth))
        
        if self.depth == self.max_depth:
            return

        self.left  = demoNode(self.max_depth)
        self.right = demoNode(self.max_depth)

        self.left.split_node(depth + 1)   # recursive call
        self.right.split_node(depth + 1)  # recursive call

### ノードクラスの動作確認

In [38]:
max_depth = 3
initial_depth = 0

In [40]:
demo_tree = demoNode(max_depth)
demo_tree.split_node(initial_depth)

Recursion depth: 0
Recursion depth: 1
Recursion depth: 2
Recursion depth: 3
Recursion depth: 3
Recursion depth: 2
Recursion depth: 3
Recursion depth: 3
Recursion depth: 1
Recursion depth: 2
Recursion depth: 3
Recursion depth: 3
Recursion depth: 2
Recursion depth: 3
Recursion depth: 3


⬆︎再帰呼び出しの呼び出された順番
1. ノード0のsplit_node()が呼ばれる。 => 0
2. ノード1_1のsplit_node()が呼ばれる。 => 1
3. ノード2_1のsplit_node()が呼ばれる。 => 2
4. ノード3_1のsplit_node()が呼ばれる。 => 3
5. ノード3_2のsplit_node()が呼ばれる。 => 3
6. ノード2_2のsplit_node()が呼ばれる。 => 2
7. ノード3_3のsplit_node()が呼ばれる。 => 3
8. ノード3_4のsplit_node()が呼ばれる。 => 3
9. ノード1_2のsplit_node()が呼ばれる。 => 1
10. ノード2_3のsplit_node()が呼ばれる。 => 2
11. ノード3_5のsplit_node()が呼ばれる。 => 3
12. ノード3_6のsplit_node()が呼ばれる。 => 3
13. ノード2_4のsplit_node()が呼ばれる。 => 2
14. ノード3_7のsplit_node()が呼ばれる。 => 3
15. ノード3_8のsplit_node()が呼ばれる。 => 3

## 決定木で使われる乱数

決定木では、各特徴量で情報利得を計算する時に乱数を使用している。

## 特徴量の重要度

決定木では、情報利得が最大になるようにノードを分割していく。

情報利得とは、ある特徴量でノードを分割した時に得られる情報量のことである。つまり、情報利得の大きさで分割する特徴量の重要度を考えることができる。

反対に重要度が低い特徴量でノードを分割すると、子ノードの不純度は高いままなので学習が中々進まない。

加えて、トレーニングサンプルに重要度が低い特徴量がある状態で最後まで決定木のノードを分割した場合、過学習に陥ってしまう可能性も高い。

トレーニングサンプルの特徴量数が膨大で学習に時間がかかる場合は、前もって少ないトレーニングサンプルで学習を実施し、各特徴量の重要度を求めておくことで、重要度が低い特徴量をトレーニングサンプルから省くことができる。

# 決定木アルゴリズムの実装

In [109]:
class Node(object):
    def __init__(self, criterion="gini", max_depth=None, random_state=None):
        self.criterion = criterion # criterion is 基準
        self.max_depth = max_depth
        self.random_state = random_state
        self.depth = None
        self.left = None
        self.right = None
        self.feature = None # このメンバ変数には、このノードにおける情報利得が最大となる特徴量のインデックス番号が入る。
        self.threshold = None # このメンバ変数には、このノードにおける情報利得が最大となるsplit_pointが入る。
        self.label = None # このメンバ変数には、このノードにおける属しているデータの個数が最も多いラベル（目的変数）が入る。
        self.impurity = None
        self.info_gain = None
        self.num_samples = None # このノードのデータの個数（サンプル数）
        self.num_classes = None
        
    def split_node(self, sample, target, depth, ini_num_classes):
        '''
        指定されたmax_depthまでの深さを持つ決定木（ノードツリー）を作成している。
        sample: 特徴量{カラム : 学習データ}
        target: 目的変数(ラベル)
        depth: ノードの深さ
        ini_num_classes: 目的変数のラベルの数
            （例）二分木ならば、ラベルの数は、0と1で二個になる。
        '''
        self.depth = depth
        self.num_samples = len(target)# このノードのデータの個数（サンプル数）を代入
        '''
        各ラベルに属するデータの個数を計測してリストで代入
       　　 （例）ラベル0のサンプル数100、ラベル1のサンプル数50
        '''
        self.num_classes = [len(target[target==i]) for i in ini_num_classes]
        
        if len(np.unique(target)) == 1:# np.unique()は、重複を取り除いた配列を返す。つまりsetを返す。
            self.label = target[0] # ラベルが1つしかないのならそもそも分類する必要はない
            self.impurity = self.criterion_func(target) # impurity is 不純物
            return
        
        '''
        辞書内包表記を使って、ディクショナリ{Key : Value}を作っている。
        Key: ラベル、Value: ラベルに属しているデータの個数
        　　（例） {ラベル0: 100, ラベル1: 50}
        '''
        class_count = {i: len(target[target==i]) for i in np.unique(target)}
        
        
        '''
        max関数のオプションkeyにlambda構文で作成したディクショナリのValueを返す無名関数を指定する。
        そうすることで、max関数はディクショナリのValueが最大になる{Key : Value}のペアを返すようになる。
        最終的にlabelには、max関数の戻り値の0番目の要素であるKeyを代入している。
        
        ちなみにこのlabelは目的変数である。
        
        今回の場合では、各ノードに含まれるラベルに属するデータの個数が最も多いラベルを返す。
        '''
        self.label = max(class_count.items(), key=lambda x:x[1])[0]

        self.impurity = self.criterion_func(target)# 不純度を計算して代入している。
        num_features = sample.shape[1]# 特徴量の数を代入している。
        self.info_gain = 0.0# 情報利得の初期化をしている。
        
        if self.random_state != None:
            np.random.seed(self.random_state)
        
        '''
        permutation関数は、引数の配列をランダムに並べ替える。
        tolist関数は、配列をリスト型に変換している。
        二分木の場合は、[0, 1]か[1, 0]になる。
        '''
        f_loop_order = np.random.permutation(num_features).tolist()
        
        '''
        特徴量ごとにループを回し、どの特徴量のどの境界(split_point)が
        このノードにおける最も高い情報利得を得ることができるかを算出する。
        '''
        for f in f_loop_order:
            uniq_feature = np.unique(sample[:, f]) # 各カラム(特徴量)のデータ一覧を重複を無くして取得している。
            
            '''
            uniq_feature[:-1]は、配列uniq_featureの最後の要素以外のすべての要素を含む配列
            uniq_feature[1:]は、配列uniq_featureの最初の要素以外のすべての要素を含む配列
              （例） uniq_feature = [1, 2, 3]とする。
                          uniq_feature[:-1] = [1, 2]、uniq_feature[1:] = [2, 3]となる。
                          よって、uniq_feature[:-1] + [1, 2]、uniq_feature[1:] = [3, 5]
                          split_points = (uniq_feature[:-1] + uniq_feature[1:]) / 2.0
                                                = [1.5, 2.5]となる。
            '''
            split_points = (uniq_feature[:-1] + uniq_feature[1:]) / 2.0# 直訳すると split_points is 分割ポイント
            
            for threshold in split_points:
                '''
                目的変数リストをleftとrightに分割している。
                split_pointによる境界でleftとrightに分けている。
                どのsplit_pointによる分割の仕方が最も高い情報利得を得ることができるかを知るために全パターンの算出結果をループで計測している。
                ''' 
                target_l = target[sample[:, f] <= threshold]
                target_r= target[sample[:, f] > threshold]
                val = self.calc_info_gain(target, target_l, target_r)# 情報利得を算出している。
                if self.info_gain < val:# より高い情報利得の場合は、以下の値を更新している。
                    self.info_gain = val # このときの情報利得の値を代入
                    self.feature = f # この情報利得の値を算出した特徴量を代入
                    self.threshold = threshold # この情報利得の値を算出したsplit_pointを代入
        
        if self.info_gain == 0.0:
            return
        if self.depth == self.max_depth:
            # この条件式を加えることで、max_depthまでの深さの決定木ノードが作られる。
            return
        
        '''
        子ノード(左)の設定
        このノードにおいて情報利得が最も高くなる特徴量とsplit_pointを使って学習データを設定する。
        　　情報利得が最も高くなる特徴量を指定して取得したデータリストがsplit_point以下のデータを設定する。
        '''
        sample_l = sample[sample[:, self.feature] <= self.threshold]
        target_l = target[sample[:, self.feature] <= self.threshold]
        self.left = Node(self.criterion, self.max_depth)
        self.left.split_node(sample_l, target_l, depth + 1, ini_num_classes) # recursive call
        
        '''
        子ノード(右)の設定
        このノードにおいて情報利得が最も高くなる特徴量とsplit_pointを使って学習データを設定する。
        　　情報利得が最も高くなる特徴量を指定して取得したデータリストがsplit_pointより大きいデータを設定する。
        '''
        sample_r = sample[sample[:, self.feature] > self.threshold]
        target_r = target[sample[:, self.feature] > self.threshold]
        self.right = Node(self.criterion, self.max_depth)
        self.right.split_node(sample_r, target_r,  depth + 1, ini_num_classes) # recursive call
    
    def criterion_func(self, target):
        '''
        メンバ変数criterionで指定された不純度で不純度を算出する
        '''
        classes = np.unique(target)
        numdata = len(target)
        
        if self.criterion == "gini":
            '''
            ジニ不純度
            IG( t ) = 1 - Σi p(i | t)^2 、(i = 1..)
            '''
            val = 1
            for c in classes:
                p = float(len(target[target == c])) / numdata # ノードtarget内でクラスcが含まれる割合を計算 p(i | t)の部分
                val -= p ** 2.0 # 1 - Σi p(i | t)^2 、(i = 1..) の部分
        elif self.criterion == "entropy":
            '''
            エントロピー
            IH( t ) = - Σi (p(i | t) * log2 p(i | t))、(i = 1..)
            '''
            val = 0
            for c in classes:
                p = float(len(target[target == c])) / numdata # ノードtarget内でクラスcが含まれる割合を計算 p(i | t)の部分
                if p != 0.0:
                    val -= p * np.log2(p)
        return val
            
    def calc_info_gain(self, target_p, target_cl, target_cr):
        '''
        情報利得の算出
        　　(親ノードの不純度) - (子ノードの不純度の総和)
        IG(Dp, f) = I(Dp) - (Nleft / Np) * I(Dleft) - (Nright / Np) * I(Dright)  
        '''
        cri_p = self.criterion_func(target_p) # I(Dp)の部分
        cri_cl = self.criterion_func(target_cl) # I(Dleft)の部分
        cri_cr = self.criterion_func(target_cr) # I(Dright)の部分
        return cri_p - len(target_cl) / float(len(target_p)) * cri_cl - len(target_cr) / float(len(target_p)) * cri_cr
    
    def predict(self, sample):
        '''
        予測結果ラベル(目的変数)を返す。
        ここで使用されているself.featureは、このノードにおいて最も高い情報利得を取得できる特徴量が格納されている。
        ここで使用されているself.thresholdは、このノードにおいて最も高い情報利得を取得できるsplit_pointが格納されている。
        '''
        if self.feature == None or self.depth == self.max_depth:
            return self.label
        else:
            if sample[self.feature] <= self.threshold:
                return self.left.predict(sample)
            else:
                return self.right.predict(sample)
        

In [122]:
class TreeAnalysis(object):
    def __init__(self):
        self.num_features = None
        self.importances = None
    
    def compute_feature_importances(self, node):
        '''
        特徴量の重要度の計算
        FI( fi ) = Σt IG(t, fi) * nt、(t ∈ Nfj)
        '''
        if node.feature == None:
            return
        
        self.importances[node.feature] += node.info_gain * node.num_samples # Σt IG(t, fi) * nt の部分
        print("分割対象の特徴量")
        print(node.feature)
        
        '''
        総和(Σt)の部分は、再帰で求めている。
        各ノードで色々な特徴量が分岐をする条件として使用されている可能性があるため、
        指定したノードからリーフノードまで再帰で呼び出し、総和を算出している。
        '''
        self.compute_feature_importances(node.left)
        self.compute_feature_importances(node.right)
    
    def get_feature_importances(self, node, num_features, normalize=True):
        '''
        特徴量の重要度の取得
        正規化をするかしないかも指定できる。
          正規化: FIn( fi ) = FI( fi ) / Σj FI( fi )、(j = 1..n)
        '''
        self.num_features = num_features
        self.importances = np.zeros(num_features) # 初期化: 最初はすべての特徴量の重要度はゼロとなる
        
        self.compute_feature_importances(node)
        
        '''
        各特徴量の重要度の値をこのノードのデータの個数で割っている。
        こうすることで、各特徴量の各ノードにおける重要度を算出する。
        '''
        self.importances /= node.num_samples 
        
        # 正規化
        if normalize:
            normalizer = np.sum(self.importances) # Σj FI( fi )の部分
            
            if normalizer > 0.0:
                # ゼロで除算しない
                self.importances /= normalizer # FI( fi ) / Σj FI( fi )の部分 
        return self.importances

In [123]:
class DecisionTree(object):
    def __init__(self, criterion="gini", max_depth=None, random_state=None):
        self.tree = None
        self.criterion = criterion
        self.max_depth = max_depth
        self.random_state = random_state
        self.tree_analysis = TreeAnalysis()
    
    def fit(self, sample, target):
        '''
        sample: 特徴量{カラム : 学習データ}
        target: 目的変数(ラベル)
        '''
        self.tree = Node(self.criterion, self.max_depth, self.random_state)
        self.tree.split_node(sample, target, 0, np.unique(target))
        self.feature_importances_ = self.tree_analysis.get_feature_importances(self.tree, sample.shape[1])
    
    def predict(self, sample):
        '''
        予測結果ラベル(目的変数)の配列を返す。
        '''
        pred = []
        for s in sample:
            pred.append(self.tree.predict(s))
        return np.array(pred)
    
    def score(self, sample, target):
        return sum(self.predict(sample) == target) / float(len(target))

# 動作確認

既存の決定木アルゴリズム（scikit-learn）と自作決定木アルゴリズムの性能比較

In [124]:
def main():
    iris = datasets.load_iris()
    X = iris.data[:,[0,2]]  # sepal length and petal length
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    max_depth    = None
    random_state = 3

    clf_m = DecisionTree(criterion="gini", max_depth=max_depth, random_state=random_state)
    clf_m.fit(X_train, y_train)
    my_score = clf_m.score(X_test, y_test)

    clf_s = tree.DecisionTreeClassifier(criterion="gini", max_depth=max_depth, random_state=random_state)
    clf_s.fit(X_train, y_train)
    sklearn_score = clf_s.score(X_test ,y_test)
    
    #--- print score
    print("-"*50)
    print("my decision tree score:" + str(my_score))
    print("scikit-learn decision tree score:" + str(sklearn_score))

    #---print feature importances
    print("-"*50)
    f_importance_m = clf_m.feature_importances_
    
    f_importance_s = clf_s.feature_importances_

    print ("my decision tree feature importances:")
    for f_name, f_importance in zip(np.array(iris.feature_names)[[0,2]], f_importance_m):
        print( "    ",f_name,":", f_importance)

    print ("sklearn decision tree feature importances:")
    for f_name, f_importance in zip(np.array(iris.feature_names)[[0,2]], f_importance_s):
        print( "    ",f_name,":", f_importance)
        

In [125]:
main()

分割対象の特徴量
1
分割対象の特徴量
1
分割対象の特徴量
1
分割対象の特徴量
0
分割対象の特徴量
1
分割対象の特徴量
0
分割対象の特徴量
0
分割対象の特徴量
0
分割対象の特徴量
1
分割対象の特徴量
0
--------------------------------------------------
my decision tree score:0.9333333333333333
scikit-learn decision tree score:0.9333333333333333
--------------------------------------------------
my decision tree feature importances:
     sepal length (cm) : 0.05572220815759174
     petal length (cm) : 0.9442777918424082
sklearn decision tree feature importances:
     sepal length (cm) : 0.05572220815759178
     petal length (cm) : 0.9442777918424082


In [61]:
iris = datasets.load_iris()
X_mh = iris.data[:,[0,2]]  # sepal length and petal length
y_mh = iris.target
X_train_mh, X_test_mh, y_train_mh, y_test_mh = train_test_split(X_mh, y_mh, test_size=0.3, random_state=0)

max_depth    = None
random_state = 3

criterion="gini"

In [102]:
#X_train_mh

In [63]:
target = y_train_mh
sample = X_train_mh
ini_num_classes = np.unique(target)

In [64]:
num_samples = len(target)
print(num_samples)

105


In [66]:
target

array([1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1,
       2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0, 0,
       2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1,
       1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 2, 1, 2, 0])

In [65]:
num_classes = [len(target[target==i]) for i in ini_num_classes]
print(num_classes)

[34, 32, 39]


In [67]:
class_count = {i: len(target[target==i]) for i in np.unique(target)}
print(class_count)

{0: 34, 1: 32, 2: 39}


In [68]:
num_features = sample.shape[1]
print(num_features)

2


In [75]:
f_loop_order = np.random.permutation(num_features).tolist()
print(f_loop_order)

[1, 0]


In [78]:
f = f_loop_order[0]
uniq_feature = np.unique(sample[:, f])
print(uniq_feature)

[1.1 1.2 1.3 1.4 1.5 1.6 1.7 3.  3.3 3.5 3.6 3.7 3.8 3.9 4.  4.1 4.2 4.3
 4.4 4.5 4.6 4.7 4.8 4.9 5.  5.1 5.2 5.3 5.4 5.5 5.6 5.7 5.8 5.9 6.  6.1
 6.4 6.6 6.7 6.9]


In [80]:
uniq_feature[:-1]

array([1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 3. , 3.3, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1,
       5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.4, 6.6, 6.7])

In [81]:
uniq_feature[1:]

array([1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 3. , 3.3, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.4, 6.6, 6.7, 6.9])

In [82]:
uniq_feature[:-1] + uniq_feature[1:]

array([ 2.3,  2.5,  2.7,  2.9,  3.1,  3.3,  4.7,  6.3,  6.8,  7.1,  7.3,
        7.5,  7.7,  7.9,  8.1,  8.3,  8.5,  8.7,  8.9,  9.1,  9.3,  9.5,
        9.7,  9.9, 10.1, 10.3, 10.5, 10.7, 10.9, 11.1, 11.3, 11.5, 11.7,
       11.9, 12.1, 12.5, 13. , 13.3, 13.6])

In [79]:
split_points = (uniq_feature[:-1] + uniq_feature[1:]) / 2.0
print(split_points)

[1.15 1.25 1.35 1.45 1.55 1.65 2.35 3.15 3.4  3.55 3.65 3.75 3.85 3.95
 4.05 4.15 4.25 4.35 4.45 4.55 4.65 4.75 4.85 4.95 5.05 5.15 5.25 5.35
 5.45 5.55 5.65 5.75 5.85 5.95 6.05 6.25 6.5  6.65 6.8 ]


In [86]:
for threshold in split_points:
        target_l = target[sample[:, f] <= threshold] 
        target_r = target[sample[:, f] >  threshold]
        print("targetの数: " + str(len(target)))
        print("target_lの数: " + str(len(target_l)))
        print("target_rの数: " + str(len(target_r)))
        print()

targetの数: 105
target_lの数: 1
target_rの数: 104

targetの数: 105
target_lの数: 3
target_rの数: 102

targetの数: 105
target_lの数: 7
target_rの数: 98

targetの数: 105
target_lの数: 16
target_rの数: 89

targetの数: 105
target_lの数: 26
target_rの数: 79

targetの数: 105
target_lの数: 31
target_rの数: 74

targetの数: 105
target_lの数: 34
target_rの数: 71

targetの数: 105
target_lの数: 35
target_rの数: 70

targetの数: 105
target_lの数: 36
target_rの数: 69

targetの数: 105
target_lの数: 38
target_rの数: 67

targetの数: 105
target_lの数: 39
target_rの数: 66

targetの数: 105
target_lの数: 40
target_rの数: 65

targetの数: 105
target_lの数: 41
target_rの数: 64

targetの数: 105
target_lの数: 43
target_rの数: 62

targetの数: 105
target_lの数: 45
target_rの数: 60

targetの数: 105
target_lの数: 48
target_rの数: 57

targetの数: 105
target_lの数: 51
target_rの数: 54

targetの数: 105
target_lの数: 52
target_rの数: 53

targetの数: 105
target_lの数: 55
target_rの数: 50

targetの数: 105
target_lの数: 59
target_rの数: 46

targetの数: 105
target_lの数: 61
target_rの数: 44

targetの数: 105
target_lの数: 63
target_rの数: 42

targetの数: 1

In [92]:
sample.shape[1]

2

In [98]:
importances = None

In [100]:
importances[1] += 0

TypeError: 'NoneType' object is not subscriptable